In [1]:
# mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
## dataloader

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler, normalize
from sklearn import datasets
def load_MSD():
# Load the raw data.
    num_attributes = 90
    names = ['Year'] + ['Attr_{}'.format(i) for i in range(num_attributes)]
    df = pd.read_csv('/content/drive/My Drive/Research/Data/YearPredictionMSD.txt', header=None, names=names)

# Validate the data.
    num_examples = 515345
    assert len(df.columns) == num_attributes + 1
    assert len(df) == num_examples
    assert not df.isnull().values.any()


# Train/test split. See "Data Set Information".
    num_train = 463715
    df = df.values
    train = df[:num_train]
    test = df[num_train:]
    del df


# Seperate inputs and outputs.
    X_train, y_train = train[:, 1:], train[:, 0]
    X_test, y_test = test[:, 1:], test[:, 0]
    del train
    del test
    
    standardize = StandardScaler().fit(X_train)
    X_train = standardize.transform(X_train)
    X_test = standardize.transform(X_test)

    y_train1 = np.expand_dims(y_train, axis=1)
    y_test1 = np.expand_dims(y_test , axis=1)

    standardize2 = StandardScaler().fit(y_train1)
    y_train = standardize2.transform(y_train1)
    y_train = np.squeeze(y_train)

    y_test1 = np.expand_dims(y_test, axis=1)
    y_test = standardize2.transform(y_test1)
    y_test = np.squeeze(y_test)
    
    return X_train, y_train, X_test, y_test


def load_boston():
    boston = datasets.load_boston()
    x = boston.data
    X = np.concatenate((x,x), axis=1)
    for i in range(5):
        X = np.concatenate((X,x), axis=1)
    X = X[:,0:-1]
    standardize = StandardScaler().fit(X)
    X = standardize.transform(X)
    return X



def load_dataset(dataset):
    if dataset == 'MSD':
        X_train, y_train, X_test, y_test = load_MSD()
        return X_train, y_train, X_test, y_test
    if dataset == 'boston':
        x = load_boston()
        return x
    
    
x = load_dataset('boston')

In [4]:
## model

import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import torch.nn.init as init
import math


__all__ = ['SDENet']

class Drift(nn.Module):
    def __init__(self):
        super(Drift, self).__init__()
        self.fc = nn.Linear(50, 50)
        self.relu = nn.ReLU(inplace=True)
    def forward(self, t, x):
        out = self.relu(self.fc(x))
        return out    



class Diffusion(nn.Module):
    def __init__(self):
        super(Diffusion, self).__init__()
        self.relu = nn.ReLU(inplace=True)
        self.fc1 = nn.Linear(50, 100)
        self.fc2 = nn.Linear(100, 1)
    def forward(self, t, x):
        out = self.relu(self.fc1(x))
        out = self.fc2(out)
        out = torch.sigmoid(out)
        return out


    
class SDENet(nn.Module):
    def __init__(self, layer_depth):
        super(SDENet, self).__init__()
        self.layer_depth = layer_depth
        self.downsampling_layers = nn.Linear(90, 50)
        self.drift = Drift()
        self.diffusion = Diffusion()
        self.fc_layers = nn.Sequential(nn.ReLU(inplace=True), nn.Linear(50, 2))
        self.deltat = 4./self.layer_depth
        self.sigma = 0.5
    def forward(self, x, training_diffusion=False):
        out = self.downsampling_layers(x)
        if not training_diffusion:
            t = 0
            diffusion_term = self.sigma*self.diffusion(t, out)
            for i in range(self.layer_depth):
                t = 4*(float(i))/self.layer_depth
                out = out + self.drift(t, out)*self.deltat + diffusion_term*math.sqrt(self.deltat)*torch.randn_like(out).to(x)

            final_out = self.fc_layers(out) 
            mean = final_out[:,0]
            sigma = F.softplus(final_out[:,1])+1e-3
            return mean, sigma
            
        else:
            t = 0
            final_out = self.diffusion(t, out.detach())  
            return final_out

def test():
    model = SDENet(layer_depth=6)
    return model  
 
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

if __name__ == '__main__':
    model = test()
    num_params = count_parameters(model)
    print(num_params)

12403


In [5]:
#  metrics

from __future__ import print_function
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import time
from scipy import misc

def tpr95(dir_name, task = 'OOD'):
    #calculate the falsepositive error when tpr is 95%
    if task == 'OOD':
        cifar = np.loadtxt('%s/confidence_Base_In.txt'%dir_name, delimiter=',')
        other = np.loadtxt('%s/confidence_Base_Out.txt'%dir_name, delimiter=',')
    elif task == 'mis':
        cifar = np.loadtxt('%s/confidence_Base_Succ.txt'%dir_name, delimiter=',')
        other = np.loadtxt('%s/confidence_Base_Err.txt'%dir_name, delimiter=',')


    Y1 = other
    X1 = cifar
    end = np.max([np.max(X1), np.max(Y1)])
    start = np.min([np.min(X1),np.min(Y1)])
    gap = (end- start)/200000 # precision:200000

    total = 0.0
    fpr = 0.0
    for delta in np.arange(start, end, gap):
        tpr = np.sum(np.sum(X1 >= delta)) / np.float(len(X1))
        error2 = np.sum(np.sum(Y1 > delta)) / np.float(len(Y1))
        if tpr <= 0.96 and tpr >= 0.94:
            fpr += error2
            total += 1
    if total == 0:
        print('corner case')
        fprBase = 1
    else:
        fprBase = fpr/total

    return fprBase


def auroc(dir_name, task = 'OOD'):
    #calculate the AUROC
    if task == 'OOD':
        f1 = open('%s/Update_Base_ROC_tpr.txt'%dir_name, 'w')
        f2 = open('%s/Update_Base_ROC_fpr.txt'%dir_name, 'w')
        cifar = np.loadtxt('%s/confidence_Base_In.txt'%dir_name, delimiter=',')
        other = np.loadtxt('%s/confidence_Base_Out.txt'%dir_name, delimiter=',')
    elif task == 'mis':
        f1 = open('%s/Update_Base_ROC_tpr_mis.txt'%dir_name, 'w')
        f2 = open('%s/Update_Base_ROC_fpr_mis.txt'%dir_name, 'w')
        cifar = np.loadtxt('%s/confidence_Base_Succ.txt'%dir_name, delimiter=',')
        other = np.loadtxt('%s/confidence_Base_Err.txt'%dir_name, delimiter=',')


    Y1 = other
    X1 = cifar
    end = np.max([np.max(X1), np.max(Y1)])
    start = np.min([np.min(X1),np.min(Y1)])
    gap = (end- start)/200000

    aurocBase = 0.0
    fprTemp = 1.0
    for delta in np.arange(start, end, gap):
        tpr = np.sum(np.sum(X1 >= delta)) / np.float(len(X1))
        fpr = np.sum(np.sum(Y1 > delta)) / np.float(len(Y1))
        f1.write("{}\n".format(tpr))
        f2.write("{}\n".format(fpr))
        aurocBase += (-fpr+fprTemp)*tpr
        fprTemp = fpr
    f1.close()
    f2.close()
    return aurocBase

def auprIn(dir_name, task ='OOD'):
    #calculate the AUPR
    if task == 'OOD':
        cifar = np.loadtxt('%s/confidence_Base_In.txt'%dir_name, delimiter=',')
        other = np.loadtxt('%s/confidence_Base_Out.txt'%dir_name, delimiter=',')
    elif task == 'mis':
        cifar = np.loadtxt('%s/confidence_Base_Succ.txt'%dir_name, delimiter=',')
        other = np.loadtxt('%s/confidence_Base_Err.txt'%dir_name, delimiter=',')

    precisionVec = []
    recallVec = []
    Y1 = other
    X1 = cifar
    end = np.max([np.max(X1), np.max(Y1)])
    start = np.min([np.min(X1),np.min(Y1)])
    gap = (end- start)/200000

    auprBase = 0.0
    recallTemp = 1.0
    for delta in np.arange(start, end, gap):
        tp = np.sum(np.sum(X1 >= delta)) #/ np.float(len(X1))
        fp = np.sum(np.sum(Y1 >= delta)) #/ np.float(len(Y1))
        if tp + fp == 0: continue
        precision = tp / (tp + fp)
        recall = tp/ np.float(len(X1))
        precisionVec.append(precision)
        recallVec.append(recall)
        auprBase += (recallTemp-recall)*precision
        recallTemp = recall
    auprBase += recall * precision

    return auprBase

def auprOut(dir_name, task = 'OOD'):
    #calculate the AUPR
    if task == 'OOD':
        cifar = np.loadtxt('%s/confidence_Base_In.txt'%dir_name, delimiter=',')
        other = np.loadtxt('%s/confidence_Base_Out.txt'%dir_name, delimiter=',')
    elif task == 'mis':
        cifar = np.loadtxt('%s/confidence_Base_Succ.txt'%dir_name, delimiter=',')
        other = np.loadtxt('%s/confidence_Base_Err.txt'%dir_name, delimiter=',')
    Y1 = other
    X1 = cifar
    end = np.max([np.max(X1), np.max(Y1)])
    start = np.min([np.min(X1),np.min(Y1)])
    gap = (end- start)/200000

    auprBase = 0.0
    recallTemp = 1.0
    for delta in np.arange(end, start, -gap):
        fp = np.sum(np.sum(X1 < delta)) #/ np.float(len(X1))
        tp = np.sum(np.sum(Y1 < delta)) #/ np.float(len(Y1))
        if tp + fp == 0: break
        precision = tp / (tp+fp)
        recall = tp/np.float(len(Y1))
        auprBase += (recallTemp-recall)*precision
        recallTemp = recall
    auprBase += recall * precision

    return auprBase

def detection(dir_name, task = 'OOD'):
    #calculate the minimum detection error
    if task == 'OOD':
        cifar = np.loadtxt('%s/confidence_Base_In.txt'%dir_name, delimiter=',')
        other = np.loadtxt('%s/confidence_Base_Out.txt'%dir_name, delimiter=',')
    elif task == 'mis':
        cifar = np.loadtxt('%s/confidence_Base_Succ.txt'%dir_name, delimiter=',')
        other = np.loadtxt('%s/confidence_Base_Err.txt'%dir_name, delimiter=',')

    Y1 = other
    X1 = cifar
    end = np.max([np.max(X1), np.max(Y1)])
    start = np.min([np.min(X1),np.min(Y1)])
    gap = (end- start)/200000

    errorBase = 1.0
    for delta in np.arange(start, end, gap):
        tpr = np.sum(np.sum(X1 < delta)) / np.float(len(X1))
        error2 = np.sum(np.sum(Y1 > delta)) / np.float(len(Y1))
        errorBase = np.minimum(errorBase, (tpr+error2)/2.0)

    return errorBase

def metric(dir_name, task):
    print("{}{:>34}".format(task, "Performance of Baseline detector"))
    fprBase = tpr95(dir_name, task)
    print("{:20}{:13.3f}%".format("TNR at TPR 95%:", (1-fprBase)*100))
    aurocBase = auroc(dir_name, task)
    print("{:20}{:13.3f}%".format("AUROC:",aurocBase*100))
    errorBase = detection(dir_name,task)
    print("{:20}{:13.3f}%".format("Detection acc:",(1-errorBase)*100))
    auprinBase = auprIn(dir_name,task)
    print("{:20}{:13.3f}%".format("AUPR In:",auprinBase*100))
    auproutBase = auprOut(dir_name,task)
    print("{:20}{:13.3f}%".format("AUPR Out:",auproutBase*100))

In [ ]:
# training

from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
# import data_loader
import os
import argparse
import numpy as np
# import models 

# parser = argparse.ArgumentParser(description='PyTorch SDENet Training')
# parser.add_argument('--epochs', type=int, default=60, help='number of epochs to train')
# parser.add_argument('--lr', default=1e-4, type=float, help='learning rate')
# parser.add_argument('--lr2', default=0.01, type=float, help='learning rate')
# parser.add_argument('--gpu', type=int, default=0)
# parser.add_argument('--seed', type=float, default=0)
# parser.add_argument('--droprate', type=float, default=0.1, help='learning rate decay')
# parser.add_argument('--decreasing_lr', default=[20], nargs='+', help='decreasing strategy')
# parser.add_argument('--decreasing_lr2', default=[], nargs='+', help='decreasing strategy')

# args = parser.parse_args()
# print(args)

class args():
  gpu = 1
  seed = 0
  lr = 1e-4
  lr2 = 0.01
  epochs = 50
  droprate = 0.1
  decreasing_lr = [20]
  decreasing_lr2 = []

# args.gpu = 1

# device = torch.device('cuda:' + str(args.gpu) if torch.cuda.is_available() else 'cpu')
device = 'cuda'

batch_size = 128
Iter = 3622
Iter_test = 403
target_scale = 10.939756
# Data
print('==> Preparing data..')



torch.manual_seed(args.seed)


if device == 'cuda':
    cudnn.benchmark = True
    torch.cuda.manual_seed(args.seed)



X_train, y_train, X_test, y_test = load_dataset('MSD')

X_train = torch.from_numpy(X_train).type(torch.FloatTensor)
X_test = torch.from_numpy(X_test).type(torch.FloatTensor)
y_train = torch.from_numpy(y_train).type(torch.FloatTensor)
y_test = torch.from_numpy(y_test).type(torch.FloatTensor)

# Model
print('==> Building model..')
net = SDENet(4)
net = net.to(device)
# net = net.to('cuda')


real_label = 0
fake_label = 1

criterion = nn.BCELoss()

optimizer_F = optim.SGD([ {'params': net.downsampling_layers.parameters()}, {'params': net.drift.parameters()},
{'params': net.fc_layers.parameters()}], lr=args.lr, momentum=0.9, weight_decay=5e-4)

optimizer_G = optim.SGD([ {'params': net.diffusion.parameters()}], lr=args.lr2, momentum=0.9, weight_decay=5e-4)

def nll_loss(y, mean, sigma):
    loss = torch.mean(torch.log(sigma**2)+(y-mean)**2/(sigma**2))
    return loss
def mse(y, mean):
    loss = torch.mean((y-mean)**2)
    return loss


def load_training(iternum):
    x = X_train[iternum*batch_size:(iternum+1)*batch_size]
    y = y_train[iternum*batch_size:(iternum+1)*batch_size]
    return x, y

def load_test(iternum):
    x = X_test[iternum*batch_size:(iternum+1)*batch_size]
    y = y_test[iternum*batch_size:(iternum+1)*batch_size]
    return x, y

# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    if epoch == 0:
        net.sigma = 0.1
    if epoch == 30:
        net.sigma = 0.5
    train_loss = 0
    train_loss_in = 0
    train_loss_out = 0
    total = 0
    for iternum in range(Iter):
        inputs, targets = load_training(iternum)
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer_F.zero_grad()
        mean, sigma = net(inputs)
        loss = nll_loss(targets, mean, sigma)
        loss.backward()
        nn.utils.clip_grad_norm_(net.parameters(), 100.)
        optimizer_F.step()
        train_loss += loss.item()

        label = torch.full((batch_size,1), real_label, device=device).to(torch.float32)
        optimizer_G.zero_grad()
        predict_in = net(inputs, training_diffusion=True).to(torch.float32)
        loss_in = criterion(predict_in, label)
        loss_in.backward()
        label.fill_(fake_label)

        inputs_out = 2*torch.randn(batch_size, 90, device = device)+inputs
        predict_out = net(inputs_out, training_diffusion=True)
        loss_out = criterion(predict_out, label)
        
        loss_out.backward()
        train_loss_out += loss_out.item()
        train_loss_in += loss_in.item()
        optimizer_G.step()
      
    print('Train epoch:{} \tLoss: {:.6f}| Loss_in: {:.6f}| Loss_out: {:.6f}'.format(epoch, train_loss/Iter, train_loss_in/Iter, train_loss_out/Iter))

def test(epoch):
    net.eval()
    test_loss = 0
    total = 0
    with torch.no_grad():
        for iternum in range(Iter_test):
            inputs, targets = load_test(iternum)
            inputs, targets = inputs.to(device), targets.to(device)
            current_mean = 0
            for i in range(10):
                mean, sigma = net(inputs)
                current_mean = current_mean + mean
            current_mean = current_mean/10
            loss = mse(targets, current_mean)*target_scale
            test_loss += loss.item()
    
    print('Test epoch:{} \tLoss: {:.6f}'.format(epoch, np.sqrt(test_loss/Iter_test)))
           



for epoch in range(0, args.epochs):
    train(epoch)
    test(epoch)
    if epoch in args.decreasing_lr:
        for param_group in optimizer_F.param_groups:
            param_group['lr'] *= args.droprate

    if epoch in args.decreasing_lr2:
        for param_group in optimizer_G.param_groups:
            param_group['lr'] *= args.droprate


if not os.path.isdir('./save_sdenet_msd'):
    os.makedirs('./save_sdenet_msd')
torch.save(net.state_dict(),'./save_sdenet_msd/final_model')

==> Preparing data..
==> Building model..

Epoch: 0
Train epoch:0 	Loss: 0.538097| Loss_in: 0.101449| Loss_out: 0.053645
Test epoch:0 	Loss: 2.848383

Epoch: 1
Train epoch:1 	Loss: 0.332331| Loss_in: 0.051145| Loss_out: 0.026649
Test epoch:1 	Loss: 2.804849

Epoch: 2
Train epoch:2 	Loss: 0.283098| Loss_in: 0.044123| Loss_out: 0.023556
Test epoch:2 	Loss: 2.778974

Epoch: 3
Train epoch:3 	Loss: 0.252933| Loss_in: 0.039622| Loss_out: 0.021917
Test epoch:3 	Loss: 2.759232

Epoch: 4
Train epoch:4 	Loss: 0.231624| Loss_in: 0.038079| Loss_out: 0.021433
Test epoch:4 	Loss: 2.743779

Epoch: 5
Train epoch:5 	Loss: 0.214659| Loss_in: 0.036348| Loss_out: 0.020486
Test epoch:5 	Loss: 2.733092

Epoch: 6
Train epoch:6 	Loss: 0.200562| Loss_in: 0.035378| Loss_out: 0.020066
Test epoch:6 	Loss: 2.725365

Epoch: 7
Train epoch:7 	Loss: 0.189228| Loss_in: 0.034552| Loss_out: 0.019780
Test epoch:7 	Loss: 2.720180

Epoch: 8
Train epoch:8 	Loss: 0.179095| Loss_in: 0.033464| Loss_out: 0.019073
Test epoch:8 	L